In [232]:
import pandas as pd
import numpy as np
import math


In [233]:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('data/ml-100k/u.user', sep='|', names=u_cols, dtype={'user_id':str})

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('data/ml-100k/u.data', sep='\t', names=r_cols, dtype={'movie_id':str, 'user_id':str})

m_cols = ['movie_id', 'title', 'release_date']
movies = pd.read_csv('data/ml-100k/u.item', sep='|', names=m_cols, usecols=range(3), encoding='latin-1', dtype={'movie_id':str})

data = pd.merge(pd.merge(ratings, users), movies)
data = data[['user_id','title', 'movie_id','rating','release_date','sex','age']]


print("BD has", str(data.shape[0]),"ratings")
print("BD has",data.user_id.nunique(),"users")
print("BD has",data.movie_id.nunique(),"movies")
data.head()


BD has 100000 ratings
BD has 943 users
BD has 1682 movies


,user_id,title,movie_id,rating,release_date,sex,age
0,196,Kolya (1996),242,3,24-Jan-1997,M,49
1,305,Kolya (1996),242,5,24-Jan-1997,M,23
2,6,Kolya (1996),242,4,24-Jan-1997,M,42
3,234,Kolya (1996),242,4,24-Jan-1997,M,60
4,63,Kolya (1996),242,3,24-Jan-1997,M,31


In [234]:
def assign_to_set(df):
    sampled_ids = np.random.choice(df.index,
                                   size=np.int64(np.ceil(df.index.size * 0.2)),
                                   replace=False)
    df.loc[sampled_ids, 'for_testing'] = True
    return df

data['for_testing'] = False
grouped = data.groupby('user_id', group_keys=False).apply(assign_to_set)
data_train = data[grouped.for_testing == False]
data_test = data[grouped.for_testing == True]
print(data_train.shape)
print(data_test.shape)
print(data_train.index & data_test.index)

print("Training data_set has", str(data_train.shape[0]),"ratings")
print("Test data set has", str(data_test.shape[0]),"ratings")
print("La BD has", data.movie_id.nunique(), "movies")


(79619, 8)
(20381, 8)
Int64Index([], dtype='int64')
Training data_set has 79619 ratings
Test data set has 20381 ratings
La BD has 1682 movies


##### How to get the set of movies from user with id  "1"?

In [284]:
def pivot_db(ratings, adjust_users=False):
    ratings_pivoted = ratings.pivot(
        index='movie_id',
        columns='user_id',
        values='rating'
    )
    if adjust_users:
        ratings_pivoted = ratings_pivoted.apply(lambda u: u-u.mean(), axis=0)
    ratings_pivoted['users_rated']=ratings_pivoted.apply(lambda m: m[-m.isnull()].index, axis=1)    
    return ratings_pivoted#.fillna(0)

In [236]:
# users = data_train_pivoted['users_rated'].iloc[0]

In [237]:
def sim_euclid(a,b):
    from scipy.spatial.distance import euclidean
    return 1.0/(1.0+euclidean(a,b))  

In [249]:
def sim_pearson(a,b):
    from scipy.spatial.distance import correlation
    return 1-correlation(a,b)

In [239]:
def sim_cosine(a,b):
    from scipy.spatial.distance import cosine
    return 1-cosine(a,b)

In [240]:
def compare(pivoted_m1, pivoted_m2, similarity):
#     m1 = df.loc[m_id1]
#     m2 = df.loc[m_id2]
    if (len(pivoted_m1)==0 or len(pivoted_m2)==0):
        return 1
    users_1 = pivoted_m1['users_rated']
    users_2 = pivoted_m2['users_rated']
    intersection = users_1[users_1.isin(users_2)]
    if len(intersection)==0: 
        return 1
    return similarity(pivoted_m1.loc[intersection], pivoted_m2.loc[intersection])

In [241]:
from tqdm import tqdm

In [286]:
class CollaborativeItemReco:
    """ Collaborative filtering using a custom sim(u,u'). """    
    def __init__(self, ratings, similarity, adjust_users=False, k=None):
        """ Constructor """        
        self.df = pivot_db(ratings, adjust_users)        
        self.similarity=similarity
        self.sim_matrix = pd.DataFrame(1, columns=self.df.index, index=self.df.index)
        self.k = k+1 if k else len(self.df.index)
    def learn(self):
        """ Prepare data structures for estimation. Similarity matrix for users """
        for i, mId1 in enumerate(tqdm(self.df.index)):
            for j, mId2 in enumerate(self.df.index[i+1:]):
                sim = compare(self.df.loc[mId1],self.df.loc[mId2], self.similarity)
                self.sim_matrix.loc[mId1,mId2]=sim
                self.sim_matrix.loc[mId2,mId1]=sim
                
    def estimate_basic(self, u, j):
        # u is user
        # j is movie
        if u not in self.df:
            print('u_{} not in training'.format(u))
            return 3
        if j not in self.sim_matrix:
            print('m_{} not in training'.format(j))
            return 3
        u_ratings = self.df[u][self.df[u]>0]
        num=0
        den=0
        P_k = self.sim_matrix[j].loc[u_ratings.index].sort_values(ascending=False).iloc[1:self.k]
        means_movies = self.df.apply(lambda m: m.loc[m['users_rated']].mean(),axis=1)
        for i, sim in P_k.iteritems():
            num+=sim*u_ratings[i]
            den+=sim
        if den==0: 
            if means_movies[j]>0:
                # return the mean movie rating if there is no similar for the computation
                return means_movies[j]
            else:
                # else return mean user rating 
                return u_ratings.mean()
        return num/den
    
    def estimate_mean(self, u, j):
        # u is user
        # j is movie
        if u not in self.df:
            print('u_{} not in training'.format(u))
            return 3
        if j not in self.sim_matrix:
            print('m_{} not in training'.format(j))
            return 3
        u_ratings = self.df[u][self.df[u]>0]
        num=0
        den=0
        P_k = self.sim_matrix[j].loc[u_ratings.index].sort_values(ascending=False).iloc[1:]
        means_movies = self.df.apply(lambda m: m.loc[m['users_rated']].mean(),axis=1)
        for i, sim in P_k.iteritems():
            r_i_mean = means_movies[i]
            num+=sim*(u_ratings[i]-r_i_mean)
            den+=sim
        if den==0: 
            if means_movies[j]>0:
                # return the mean movie rating if there is no similar for the computation
                return means_movies[j]
            else:
                # else return mean user rating 
                return u_ratings.mean()
        return means_movies[j]+num/den

In [299]:
data_train_sample = data_train[data_train.movie_id.apply(lambda x: int(x)<100)]
data_test_sample = data_test[data_test.movie_id.apply(lambda x: int(x)<100)]

In [277]:
item_cosine_sample = CollaborativeItemReco(data_train_sample, sim_cosine)
item_cosine_sample.learn()


 66%|█████████████████████████████████████████████████████▊                            | 65/99 [00:22<00:06,  5.38it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 99/99 [00:25<00:00,  3.90it/s]


In [282]:
item_cosine_sample.estimate_mean("1","1")

4.1127366135337615

In [221]:
# data_test = data_test[data_test.movie_id.apply(lambda x: int(x)<100)]

In [243]:
item_cosine = CollaborativeItemReco(data_train, sim_cosine)
item_cosine.learn()


  4%|███                                                                           | 65/1653 [06:55<1:56:34,  4.40s/it]


  8%|██████                                                                       | 131/1653 [11:44<1:56:08,  4.58s/it]


 12%|█████████▏                                                                   | 197/1653 [16:07<1:32:03,  3.79s/it]


 16%|████████████▎                                                                | 263/1653 [20:21<1:20:36,  3.48s/it]


 20%|███████████████▎                                                             | 329/1653 [24:03<1:01:24,  2.78s/it]


...

 84%|█████████████████████████████████████████████████████████████████▎            | 1385/1653 [58:02<03:13,  1.39it/s]


 88%|████████████████████████████████████████████████████████████████████▍         | 1451/1653 [58:43<01:46,  1.89it/s]


 92%|███████████████████████████████████████████████████████████████████████▌      | 1517/1653 [59:13<00:51,  2.64it/s]


 96%|██████████████████████████████████████████████████████████████████████████▋   | 1583/1653 [59:31<00:13,  5.12it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 1653/1653 [59:38<00:00,  2.16s/it]


In [244]:
item_cosine.sim_matrix.to_pickle('cosine.pkl')

In [287]:
item_cosine_adjusted = CollaborativeItemReco(data_train, sim_cosine, adjust_users=True)
item_cosine_adjusted.learn()


  0%|                                                                                         | 0/1653 [00:00<?, ?it/s]C:\Users\berna\Anaconda3\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)

  4%|███                                                                           | 64/1653 [12:12<4:44:19, 10.74s/it]


  8%|██████                                                                       | 130/1653 [22:56<3:49:52,  9.06s/it]


 12%|█████████▏                                                                   | 196/1653 [32:44<3:19:17,  8.21s/it]


 16%|████████████▏                                                                | 262/1653 [42:01<3:16:26,  8.47s/it]


 20%|███████████████▎                                                             | 328/1653 [50:15<2:14:31,  6.09s/it]


...

 84%|███████████████████████████████████████████████████████████████▋            | 1384/1653 [2:06:35<07:20,  1.64s/it]


 88%|██████████████████████████████████████████████████████████████████▋         | 1450/1653 [2:08:14<04:29,  1.33s/it]


 92%|█████████████████████████████████████████████████████████████████████▋      | 1516/1653 [2:09:24<02:00,  1.14it/s]


 96%|████████████████████████████████████████████████████████████████████████▋   | 1582/1653 [2:10:06<00:30,  2.34it/s]


100%|████████████████████████████████████████████████████████████████████████████| 1653/1653 [2:10:21<00:00,  4.73s/it]


In [288]:
item_cosine_adjusted.sim_matrix.to_pickle('cosine_adjusted.pkl')

In [252]:
item_pearson = CollaborativeItemReco(data_train, sim_pearson)
item_pearson.learn()


  0%|                                                                                         | 0/1653 [00:00<?, ?it/s]C:\Users\berna\Anaconda3\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)

  4%|███                                                                           | 64/1653 [05:33<2:00:12,  4.54s/it]


  8%|██████                                                                       | 130/1653 [10:22<1:49:51,  4.33s/it]


 12%|█████████▏                                                                   | 196/1653 [14:49<1:32:34,  3.81s/it]


 16%|████████████▏                                                                | 262/1653 [19:10<1:25:22,  3.68s/it]


 20%|███████████████▎                                                             | 328/1653 [22:58<1:03:36,  2.88s/it]


...

 84%|█████████████████████████████████████████████████████████████████▎            | 1384/1653 [57:49<03:17,  1.36it/s]


 88%|████████████████████████████████████████████████████████████████████▍         | 1450/1653 [58:32<01:55,  1.76it/s]


 92%|███████████████████████████████████████████████████████████████████████▌      | 1516/1653 [59:02<00:54,  2.51it/s]


 96%|██████████████████████████████████████████████████████████████████████████▋   | 1582/1653 [59:21<00:15,  4.65it/s]


100%|██████████████████████████████████████████████████████████████████████████████| 1653/1653 [59:28<00:00,  2.16s/it]


In [253]:
item_pearson.sim_matrix.to_pickle('pearson.pkl')

In [289]:
item_pearson_adjusted = CollaborativeItemReco(data_train, sim_pearson, adjust_users=True)
item_pearson_adjusted.learn()


  0%|                                                                                         | 0/1653 [00:00<?, ?it/s]C:\Users\berna\Anaconda3\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)

  4%|███                                                                           | 64/1653 [09:04<2:04:56,  4.72s/it]


  8%|██████                                                                       | 130/1653 [13:58<1:49:51,  4.33s/it]


 12%|█████████▏                                                                   | 196/1653 [18:29<1:33:26,  3.85s/it]


 16%|████████████▏                                                                | 262/1653 [22:53<1:28:52,  3.83s/it]


 20%|███████████████▎                                                             | 328/1653 [26:44<1:04:16,  2.91s/it]


...

 84%|███████████████████████████████████████████████████████████████▋            | 1384/1653 [1:02:30<03:19,  1.35it/s]


 88%|██████████████████████████████████████████████████████████████████▋         | 1450/1653 [1:03:14<01:56,  1.74it/s]


 92%|█████████████████████████████████████████████████████████████████████▋      | 1516/1653 [1:03:45<00:54,  2.49it/s]


 96%|████████████████████████████████████████████████████████████████████████▋   | 1582/1653 [1:04:04<00:15,  4.52it/s]


100%|████████████████████████████████████████████████████████████████████████████| 1653/1653 [1:04:11<00:00,  2.33s/it]


In [290]:
item_pearson_adjusted.sim_matrix.to_pickle('pearson_adjusted.pkl')

In [254]:
item_euclid = CollaborativeItemReco(data_train, sim_euclid)
item_euclid.learn()


  4%|███                                                                           | 65/1653 [06:44<2:28:16,  5.60s/it]


  8%|██████                                                                       | 131/1653 [11:58<1:48:42,  4.29s/it]


 12%|█████████▏                                                                   | 197/1653 [16:27<1:34:19,  3.89s/it]


 16%|████████████▎                                                                | 263/1653 [20:49<1:23:24,  3.60s/it]


 20%|███████████████▎                                                             | 329/1653 [24:35<1:03:35,  2.88s/it]


...

 84%|█████████████████████████████████████████████████████████████████▎            | 1385/1653 [59:41<03:14,  1.38it/s]


 88%|██████████████████████████████████████████████████████████████████▋         | 1451/1653 [1:00:23<01:48,  1.87it/s]


 92%|█████████████████████████████████████████████████████████████████████▋      | 1517/1653 [1:00:53<00:51,  2.64it/s]


 96%|████████████████████████████████████████████████████████████████████████▊   | 1583/1653 [1:01:11<00:14,  4.91it/s]


100%|████████████████████████████████████████████████████████████████████████████| 1653/1653 [1:01:18<00:00,  2.23s/it]


In [255]:
item_euclid.sim_matrix.to_pickle('euclid.pkl')

In [225]:
# data

In [226]:
def compute_rmse(y_pred, y_true):
    """ Compute Root Mean Squared Error. """
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

In [297]:
def evaluate(estimate_f,data_train,data_test):
    """ RMSE-based predictive performance evaluation with pandas. """
    u_train = set(data_train.user_id)
    m_train = set(data_train.movie_id)
    estimated = np.array([estimate_f(u,i) for (u,i) in tqdm(zip(tqdm(data_test.user_id), data_test.movie_id))])
    real = data_test.rating.values
    return compute_rmse(estimated, real)

In [291]:
all_models = {'cosine':item_cosine, 'cosine_adj':item_cosine_adjusted, 'pearson':item_pearson, 'pearson_adj':item_pearson_adjusted, 'euclid':item_euclid}

In [293]:
data_test.to_pickle('data_test.pkl')
data_train.to_pickle('data_train.pkl')

In [303]:
rmse = []
for model_label, model in all_models.items():
    print('--------- {} ---------'.format(model_label))
    rmse.append(evaluate(model.estimate_basic, data_train, data_test.sample(1000)))
    print('RMSE basic of {}: {}'.format(model_label,rmse[-1]))
    rmse.append(evaluate(model.estimate_mean, data_train, data_test.sample(1000)))
    print('RMSE mean of {}: {}'.format(model_label,rmse[-1]))







  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]






0it [00:00, ?it/s]

--------- cosine ---------









1it [00:02,  2.11s/it]





  0%|                                                                                 | 1/1000 [00:02<35:18,  2.12s/it]






2it [00:04,  2.10s/it]





  0%|▏                                                                                | 2/1000 [00:04<35:01,  2.11s/it]






3it [00:06,  2.11s/it]





  0%|▏                                                                                | 3/1000 [00:06<35:07,  2.11s/it]






4it [00:08,  2.09s/it]





  0%|▎                                                                                | 4/1000 [00:08<34:46,  2.09s/it]






5it [00:10,  2.11s/it]





  0%|▍                                                                                | 5/1000 [00:10<35:02,  2.11s/it]






6it [00:12,  2.11s/it]





  1%|▍                                                                                | 6/1000 [00:12<34:56,  2.11s/it]






7it [00:14,  2.10s/it]





  1%|▌                                   

  4%|███▍                                                                            | 43/1000 [01:30<33:57,  2.13s/it]






44it [01:32,  2.13s/it]





  4%|███▌                                                                            | 44/1000 [01:32<33:58,  2.13s/it]






45it [01:34,  2.14s/it]





  4%|███▌                                                                            | 45/1000 [01:34<34:07,  2.14s/it]






46it [01:36,  2.12s/it]





  5%|███▋                                                                            | 46/1000 [01:36<33:47,  2.12s/it]






47it [01:38,  2.13s/it]





  5%|███▊                                                                            | 47/1000 [01:38<33:45,  2.13s/it]






48it [01:40,  2.14s/it]





  5%|███▊                                                                            | 48/1000 [01:40<34:01,  2.14s/it]






49it [01:43,  2.14s/it]





  5%|███▉                                                             

  8%|██████▊                                                                         | 85/1000 [02:59<32:02,  2.10s/it]






86it [03:01,  2.09s/it]





  9%|██████▉                                                                         | 86/1000 [03:01<31:49,  2.09s/it]






87it [03:03,  2.09s/it]





  9%|██████▉                                                                         | 87/1000 [03:03<31:50,  2.09s/it]






88it [03:05,  2.10s/it]





  9%|███████                                                                         | 88/1000 [03:05<31:55,  2.10s/it]






89it [03:07,  2.09s/it]





  9%|███████                                                                         | 89/1000 [03:07<31:45,  2.09s/it]






90it [03:09,  2.10s/it]





  9%|███████▏                                                                        | 90/1000 [03:09<31:48,  2.10s/it]






91it [03:11,  2.10s/it]





  9%|███████▎                                                         

 13%|██████████                                                                     | 127/1000 [04:27<30:24,  2.09s/it]






128it [04:29,  2.07s/it]





 13%|██████████                                                                     | 128/1000 [04:29<30:06,  2.07s/it]






129it [04:31,  2.08s/it]





 13%|██████████▏                                                                    | 129/1000 [04:31<30:10,  2.08s/it]






130it [04:33,  2.09s/it]





 13%|██████████▎                                                                    | 130/1000 [04:33<30:14,  2.09s/it]






131it [04:35,  2.10s/it]





 13%|██████████▎                                                                    | 131/1000 [04:35<30:21,  2.10s/it]






132it [04:37,  2.10s/it]





 13%|██████████▍                                                                    | 132/1000 [04:37<30:21,  2.10s/it]






133it [04:40,  2.11s/it]





 13%|██████████▌                                                

KeyboardInterrupt: 

In [300]:
evaluate(item_cosine_sample.estimate_basic, data_train_sample, data_test_sample)



  0%|                                                                                         | 0/2926 [00:00<?, ?it/s]


0it [00:00, ?it/s]


1it [00:00,  8.45it/s]

  0%|                                                                                 | 1/2926 [00:00<06:11,  7.87it/s]


2it [00:00,  8.09it/s]

  0%|                                                                                 | 2/2926 [00:00<06:20,  7.69it/s]


3it [00:00,  8.26it/s]

  0%|                                                                                 | 3/2926 [00:00<06:07,  7.95it/s]


4it [00:00,  7.58it/s]

  0%|                                                                                 | 4/2926 [00:00<06:35,  7.38it/s]


5it [00:00,  7.61it/s]

  0%|▏                                                                                | 5/2926 [00:00<06:28,  7.52it/s]


6it [00:00,  7.84it/s]

  0%|▏                                                                                | 6/2926 [00:00

  2%|█▎                                                                              | 50/2926 [00:08<08:10,  5.86it/s]


51it [00:08,  5.73it/s]

  2%|█▍                                                                              | 51/2926 [00:08<08:22,  5.72it/s]


52it [00:08,  6.00it/s]

  2%|█▍                                                                              | 52/2926 [00:08<08:02,  5.96it/s]


53it [00:09,  5.07it/s]

  2%|█▍                                                                              | 53/2926 [00:09<09:36,  4.98it/s]


54it [00:09,  5.45it/s]

  2%|█▍                                                                              | 54/2926 [00:09<08:46,  5.46it/s]


55it [00:09,  4.72it/s]

  2%|█▌                                                                              | 55/2926 [00:09<10:11,  4.69it/s]


56it [00:09,  4.32it/s]

  2%|█▌                                                                              | 56/2926 [00:09<11:00,  4.34it/s

u_520 not in training


  2%|█▋                                                                              | 61/2926 [00:10<08:57,  5.33it/s]


62it [00:10,  5.67it/s]

  2%|█▋                                                                              | 62/2926 [00:10<08:20,  5.72it/s]


63it [00:10,  6.33it/s]

  2%|█▋                                                                              | 63/2926 [00:10<07:31,  6.33it/s]


64it [00:11,  6.28it/s]

  2%|█▋                                                                              | 64/2926 [00:11<07:32,  6.32it/s]


65it [00:11,  6.87it/s]

  2%|█▊                                                                              | 65/2926 [00:11<06:57,  6.85it/s]


66it [00:11,  6.67it/s]

  2%|█▊                                                                              | 66/2926 [00:11<07:09,  6.65it/s]


67it [00:11,  6.68it/s]

  2%|█▊                                                                              | 67/2926 [00:11<07:06,  6.70it/s

  4%|███                                                                            | 112/2926 [00:18<06:31,  7.18it/s]


113it [00:18,  7.33it/s]

  4%|███                                                                            | 113/2926 [00:18<06:21,  7.37it/s]


114it [00:18,  7.52it/s]

  4%|███                                                                            | 114/2926 [00:18<06:13,  7.52it/s]


115it [00:18,  7.35it/s]

  4%|███                                                                            | 115/2926 [00:18<06:24,  7.31it/s]


116it [00:18,  7.25it/s]

  4%|███▏                                                                           | 116/2926 [00:18<06:28,  7.24it/s]


117it [00:18,  7.01it/s]

  4%|███▏                                                                           | 117/2926 [00:18<06:39,  7.03it/s]


118it [00:18,  7.46it/s]

  4%|███▏                                                                           | 118/2926 [00:18<06:15,  7.

  6%|████▎                                                                          | 162/2926 [00:24<06:22,  7.22it/s]


163it [00:25,  7.06it/s]

  6%|████▍                                                                          | 163/2926 [00:25<06:30,  7.08it/s]


164it [00:25,  7.60it/s]

  6%|████▍                                                                          | 164/2926 [00:25<06:02,  7.62it/s]


165it [00:25,  7.86it/s]

  6%|████▍                                                                          | 165/2926 [00:25<05:50,  7.87it/s]


166it [00:25,  7.55it/s]

  6%|████▍                                                                          | 166/2926 [00:25<06:05,  7.55it/s]


167it [00:25,  7.86it/s]

  6%|████▌                                                                          | 167/2926 [00:25<05:53,  7.81it/s]


168it [00:25,  7.64it/s]

  6%|████▌                                                                          | 168/2926 [00:25<06:00,  7.

  7%|█████▋                                                                         | 212/2926 [00:31<06:24,  7.06it/s]


213it [00:31,  6.98it/s]

  7%|█████▊                                                                         | 213/2926 [00:31<06:29,  6.96it/s]


214it [00:32,  6.80it/s]

  7%|█████▊                                                                         | 214/2926 [00:32<06:38,  6.81it/s]


215it [00:32,  6.61it/s]

  7%|█████▊                                                                         | 215/2926 [00:32<06:49,  6.62it/s]


216it [00:32,  6.83it/s]

  7%|█████▊                                                                         | 216/2926 [00:32<06:34,  6.87it/s]


217it [00:32,  7.23it/s]

  7%|█████▊                                                                         | 217/2926 [00:32<06:14,  7.23it/s]


218it [00:32,  7.39it/s]

  7%|█████▉                                                                         | 218/2926 [00:32<06:05,  7.

  9%|███████▏                                                                       | 264/2926 [00:40<06:34,  6.74it/s]


265it [00:40,  7.27it/s]

  9%|███████▏                                                                       | 265/2926 [00:40<06:05,  7.28it/s]


266it [00:40,  6.62it/s]

  9%|███████▏                                                                       | 266/2926 [00:40<06:42,  6.62it/s]


267it [00:40,  6.42it/s]

  9%|███████▏                                                                       | 267/2926 [00:40<06:54,  6.42it/s]


268it [00:40,  6.84it/s]

  9%|███████▏                                                                       | 268/2926 [00:40<06:30,  6.81it/s]


269it [00:40,  7.07it/s]

  9%|███████▎                                                                       | 269/2926 [00:40<06:15,  7.08it/s]


270it [00:41,  7.30it/s]

  9%|███████▎                                                                       | 270/2926 [00:41<06:02,  7.

KeyboardInterrupt: 

In [295]:
print('RMSE for Collaborative Recomender: %s' % evaluate(item_cosine.estimate_mean,data_train,data_test))


  0%|▏                                                                           | 47/20381 [01:33<10:57:37,  1.94s/it]

KeyboardInterrupt: 

In [230]:
print('RMSE for Collaborative Recomender: %s' % evaluate(item_cosine.estimate_basic,data_train,data_test))

  2%|█▍                                                                              | 52/2883 [00:07<06:26,  7.32it/s]

u_520 not in training


 42%|████████████████████████████████▋                                             | 1208/2883 [02:56<03:05,  9.03it/s]

u_507 not in training
u_509 not in training


 57%|████████████████████████████████████████████                                  | 1630/2883 [03:53<02:08,  9.76it/s]

u_888 not in training


 62%|████████████████████████████████████████████████                              | 1776/2883 [04:18<02:04,  8.87it/s]

u_285 not in training


 64%|█████████████████████████████████████████████████▌                            | 1834/2883 [04:28<03:03,  5.72it/s]

KeyboardInterrupt: 

In [216]:
def precision_recall(estimate_f, data_train, data_test, N=25):
    all_movies_ids = set(data_train.movie_id.values).union(set(data_train.movie_id.values))
    in_top = 0
    ratings_5_test = data_test[data_test.rating==5]
    for i, row in ratings_5_test.iterrows():
        user_seen = list(data_train[data_train.user_id==row.user_id].movie_id.values)+list(data_test[data_test.user_id==row.user_id].movie_id.values)
        unseen = all_movies_ids.difference(user_seen)
        choosen_unseen = np.random.choice(list(unseen), min(len(unseen), 100), replace=False)
        ranked_random = pd.Series(list(map(lambda i:estimate_f(u=row.user_id,i=i),choosen_unseen))+[estimate_f(row.user_id, row.movie_id)],index=list(choosen_unseen)+[row.movie_id])
        index_row = np.argwhere(ranked_random.sort_values(ascending=False).index.values==row.movie_id).flatten()[0]
        if index_row<25:
            in_top+=1
    return in_top/len(ratings_5_test)
#     print(row.user_id)
#     break

In [107]:
precision_recall(item_cosine.estimate_basic,data_train,data_test)

279


In [218]:
len(data_train)

11501

In [188]:
# data_test, data_train


In [169]:
def rank(u,i):
    return np.sum([int(c) for c in i])

Index(['98', '97', '69', '78', '77', '86', '85', '76', '58', '49', '57', '84',
       '75', '39', '93', '48', '74', '38', '19', '37', '82', '91', '55', '46',
       '72', '9', '54', '36', '45', '18', '8', '35', '26', '53', '25', '43',
       '34', '15', '6', '51', '14', '23', '5', '3', '20', '11'],
      dtype='object')

34

In [177]:
np.where(ranked_random.sort_values(ascending=False).index==row.movie_id)

array([[37],
       [38]], dtype=int64)

In [ ]:
data_train.user_id

In [44]:
for u,i in ids_to_estimate:
    print(u,i, str(u) in data_train.user_id)

92 67 False
110 67 False
184 67 False
326 67 False
453 67 False
622 67 False
643 67 False
684 67 False
712 67 False
727 67 False
741 67 False
833 67 False
868 67 False
892 67 False
196 25 False
157 25 False
99 25 False
59 25 False
243 25 False
222 25 False
279 25 False
145 25 False
90 25 False
271 25 False
44 25 False
264 25 False
268 25 False
256 25 False
15 25 False
207 25 False
14 25 False
193 25 False
78 25 False
185 25 False
159 25 False
24 25 False
277 25 False
343 25 False
354 25 False
406 25 False
417 25 False
424 25 False
450 25 False
452 25 False
453 25 False
458 25 False
503 25 False
504 25 False
518 25 False
525 25 False
520 25 False
517 25 False
533 25 False
537 25 False
560 25 False
577 25 False
555 25 False
588 25 False
614 25 False
625 25 False
632 25 False
636 25 False
672 25 False
692 25 False
703 25 False
765 25 False
790 25 False
823 25 False
835 25 False
843 25 False
848 25 False
852 25 False
896 25 False
897 25 False
899 25 False
933 25 False
936 25 False
194 94 F

640 96 False
645 96 False
648 96 False
659 96 False
661 96 False
698 96 False
709 96 False
712 96 False
758 96 False
753 96 False
774 96 False
796 96 False
807 96 False
815 96 False
830 96 False
826 96 False
867 96 False
871 96 False
883 96 False
889 96 False
901 96 False
913 96 False
932 96 False
301 29 False
246 29 False
13 29 False
267 29 False
11 29 False
110 29 False
1 29 False
56 29 False
158 29 False
197 29 False
109 29 False
313 29 False
328 29 False
387 29 False
495 29 False
541 29 False
545 29 False
543 29 False
642 29 False
650 29 False
660 29 False
796 29 False
798 29 False
830 29 False
846 29 False
864 29 False
881 29 False
896 29 False
286 85 False
291 85 False
308 85 False
10 85 False
293 85 False
49 85 False
327 85 False
453 85 False
476 85 False
551 85 False
640 85 False
690 85 False
751 85 False
788 85 False
804 85 False
916 85 False
308 24 False
178 24 False
251 24 False
279 24 False
54 24 False
159 24 False
314 24 False
318 24 False
363 24 False
371 24 False
374 24 

592 22 False
600 22 False
618 22 False
615 22 False
638 22 False
682 22 False
683 22 False
686 22 False
694 22 False
737 22 False
788 22 False
823 22 False
826 22 False
833 22 False
881 22 False
916 22 False
933 22 False
286 72 False
200 72 False
102 72 False
42 72 False
216 72 False
49 72 False
307 72 False
144 72 False
159 72 False
327 72 False
328 72 False
336 72 False
343 72 False
363 72 False
389 72 False
393 72 False
406 72 False
716 72 False
727 72 False
848 72 False
881 72 False
889 72 False
892 72 False
887 72 False
921 72 False
934 72 False
943 72 False
6 69 False
62 69 False
210 69 False
276 69 False
7 69 False
42 69 False
267 69 False
90 69 False
271 69 False
198 69 False
41 69 False
269 69 False
5 69 False
77 69 False
187 69 False
184 69 False
144 69 False
65 69 False
235 69 False
288 69 False
188 69 False
311 69 False
106 69 False
312 69 False
318 69 False
327 69 False
331 69 False
339 69 False
343 69 False
407 69 False
416 69 False
419 69 False
456 69 False
487 69 False


305 48 False
291 48 False
308 48 False
201 48 False
13 48 False
269 48 False
18 48 False
1 48 False
296 48 False
65 48 False
221 48 False
156 48 False
106 48 False
393 48 False
406 48 False
458 48 False
478 48 False
487 48 False
533 48 False
556 48 False
561 48 False
577 48 False
615 48 False
655 48 False
716 48 False
747 48 False
741 48 False
851 48 False
883 48 False
903 48 False
916 48 False
305 83 False
122 83 False
299 83 False
249 83 False
292 83 False
11 83 False
18 83 False
151 83 False
70 83 False
307 83 False
297 83 False
158 83 False
270 83 False
187 83 False
235 83 False
361 83 False
393 83 False
409 83 False
417 83 False
429 83 False
435 83 False
488 83 False
500 83 False
536 83 False
642 83 False
648 83 False
716 83 False
747 83 False
746 83 False
741 83 False
798 83 False
815 83 False
840 83 False
899 83 False
913 83 False
253 87 False
59 87 False
87 87 False
42 87 False
174 87 False
56 87 False
16 87 False
374 87 False
409 87 False
416 87 False
465 87 False
496 87 False

<div class  = "alert alert-success">**EXERCISE 3**<p>
Modify the similarity function with the following:
$$new\_sim(a,b) = sim(a,b) * \frac{min(50,|P_{ab}|)}{50} $$
where $|P_{ab}|$ is the number of common items with user $a$ and user $b$
</div>

<div class  = "alert alert-success">**EXERCISE 4**<p>
Is there a set of users where the systems work better than with othes users?
Does it depend on the number of rating per user?
</div>